In [6]:
import pandas as pd


In [7]:
import lstm
from lstm import PortfolioDataset, HybridPortfolioModel

In [8]:
benchmark_weights = {
    'Materials': 0.025,
    'Industrials': 0.10,
    'Consumer Discretionary': 0.10,
    'Consumer Staples': 0.05,
    'Health Care': 0.10,
    'Financials': 0.15,
    'Info Tech': 0.30,
    'Comm Services': 0.10,
    'Utilities': 0.025,
    'Real Estate': 0.025,
    'Energy': 0.025
}

# Portfolio Dataset

In [9]:
pfd = PortfolioDataset(data_path="data.csv", window_size=252, portfolio_columns=lstm.sp500_sectors.values())

In [10]:
features, returns = pfd.prepare_sequence(start_idx=0)

In [11]:
features

tensor([[ 1.7000e-03, -2.0000e-04, -6.0000e-04,  ..., -8.1222e-03,
          2.3355e-02,  8.2359e-03],
        [ 3.0000e-04,  2.6000e-03, -8.1000e-03,  ..., -5.3881e-03,
         -1.1004e-02, -2.1805e-02],
        [-1.0000e-04,  2.1000e-03, -3.7000e-03,  ...,  6.9893e-04,
          4.3216e-04,  1.0194e-02],
        ...,
        [ 1.5000e-02, -7.7000e-03,  3.2000e-03,  ..., -2.6890e-04,
         -1.7937e-02, -1.1994e-02],
        [ 1.0900e-02, -3.0000e-04, -5.6000e-03,  ...,  7.3943e-04,
         -4.2025e-03,  1.7899e-03],
        [-2.0000e-03, -4.5000e-03, -7.3000e-03,  ..., -1.0130e-02,
         -2.8412e-03, -5.2607e-04]])

In [12]:
returns

tensor([ 0.0313,  0.0371, -0.0287,  0.0374, -0.0318, -0.0008,  0.0757,  0.0622,
         0.0980, -0.0205,  0.0313])

In [13]:
features, returns = pfd.get_prediction_data()

In [14]:
features

tensor([[ 0.0028, -0.0013, -0.0029,  ...,  0.0053, -0.0064, -0.0029],
        [ 0.0020, -0.0021, -0.0104,  ...,  0.0017, -0.0067,  0.0062],
        [ 0.0094,  0.0075,  0.0005,  ..., -0.0099,  0.0023,  0.0063],
        ...,
        [-0.0109,  0.0024,  0.0074,  ...,  0.0041, -0.0051, -0.0051],
        [-0.0109,  0.0024,  0.0074,  ...,  0.0041, -0.0051, -0.0051],
        [-0.0109,  0.0024,  0.0074,  ...,  0.0041, -0.0051, -0.0051]])

In [15]:
returns

tensor([0.0831, 0.1708, 0.2651, 0.1506, 0.0128, 0.2905, 0.2473, 0.3160, 0.2771,
        0.1057, 0.0708])

# Hybrid Portfolio Model

In [16]:
hpm =  HybridPortfolioModel(pfd.state_size, 64, pfd.num_assets, benchmark_weights=benchmark_weights)

In [18]:
hpm.hidden_dim

64

In [19]:
hpm.num_layers

2

In [20]:
hpm.lstm

LSTM(48, 64, num_layers=2, batch_first=True, dropout=0.2)

In [21]:
hpm.allocation

Sequential(
  (0): Linear(in_features=64, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=11, bias=True)
  (3): Softmax(dim=-1)
  (4): MinMaxScaler()
)

In [32]:
hpm.forward(pfd.prepare_sequence(850)[0].unsqueeze(0))

tensor([[0.0700, 0.0800, 0.0900, 0.0700, 0.0700, 0.0900, 0.2000, 0.0900, 0.0800,
         0.0900, 0.0900]], grad_fn=<DivBackward0>)

# Portfolio Optimizer

In [17]:
# optimizer = PortfolioOptimizer(dataset, hidden_dim=64, lr=0.0005)
